In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
import pyodbc

In [31]:
#Create a connection to my database
#DATABASE DETAILS
server = "ASUS"
password = "Campusx@21"

#TABLE_NAMES
table_name = ["company_dim2","job_postings_fact2","skills_dim2","skills_job_dim2"]

In [42]:
import pyodbc
import warnings

warnings.filterwarnings("ignore", message=".*pandas only supports SQLAlchemy.*")

conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=ASUS;"  # or ASUS\\SQLEXPRESS if named instance
    "DATABASE=class;"
    "UID=sa;"
    "PWD=Campusx@21;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)
table_names = ["company_dim2","job_postings_fact2","skills_dim2","skills_job_dim2"]
conn = pyodbc.connect(conn_str)
print(" Connected to database!")

 Connected to database!


In [40]:
#Dicitionary to store all the tables
dataframes = {}

In [43]:
for table_name in table_names:
    query = f"select * from {table_name}"
    df = pd.read_sql(query,conn)
    dataframes[table_name] = df

conn.close()

In [47]:
company = dataframes['company_dim2']
Job_posting = dataframes['job_postings_fact2']
skill_id = dataframes['skills_job_dim2']
skills = dataframes['skills_dim2']

In [54]:
merge = pd.merge(company,Job_posting, on="company_id",how = "left")

In [56]:
 merge = pd.merge(merge,skill_id, on="job_id",how = "left")

In [58]:
merge = pd.merge(merge,skills, on="skill_id",how = "left")

In [60]:
merge.columns

Index(['company_id', 'name', 'link', 'link_google', 'thumbnail', 'job_id',
       'job_title_short', 'job_title', 'job_location', 'job_via',
       'job_schedule_type', 'job_work_from_home', 'search_location',
       'job_posted_date', 'job_no_degree_mention', 'job_health_insurance',
       'job_country', 'salary_rate', 'salary_year_avg', 'salary_hour_avg',
       'skill_id', 'skills', 'type'],
      dtype='object')

In [61]:
columns_to_be_dropped = ['link', 'link_google', 'thumbnail','job_via','job_no_degree_mention', 'job_health_insurance',
       'job_country', 'salary_rate', 'salary_year_avg', 'salary_hour_avg',
       'skill_id','type']

In [62]:
merge.drop(columns = columns_to_be_dropped,inplace=True)

In [63]:
merge.columns

Index(['company_id', 'name', 'job_id', 'job_title_short', 'job_title',
       'job_location', 'job_schedule_type', 'job_work_from_home',
       'search_location', 'job_posted_date', 'skills'],
      dtype='object')

In [64]:
merge['skills']= merge['skills'].fillna('').astype(str)

In [66]:
final_dataframe = merge.groupby(['company_id', 'name', 'job_id', 'job_title_short', 'job_title',
       'job_location', 'job_schedule_type', 'job_work_from_home',
       'search_location', 'job_posted_date'])['skills'].agg(','.join).reset_index()

In [67]:
final_dataframe

,company_id,name,job_id,job_title_short,job_title,job_location,job_schedule_type,job_work_from_home,search_location,job_posted_date,skills
0,0,Cryptology,0,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,True,Serbia,2023-09-25 17:46:06,"sql,python"
1,0,Cryptology,1695656,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,True,United Kingdom,2023-09-25 18:11:19,
2,0,Cryptology,1819517,Data Analyst,Marketing Data Analyst,"Madrid, Spain",Full-time,False,Spain,2023-10-10 10:32:19,"sql,python"
3,0,Cryptology,923232,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,True,Spain,2023-09-25 17:51:21,"sql,python"
4,1,Edraak,426775,Cloud Engineer,Virtualization Engineer,"Kuwait City, Kuwait",Full-time,False,Kuwait,2023-06-06 17:35:59,
...,...,...,...,...,...,...,...,...,...,...,...
773931,99997,1st Source Bank,1035725,Data Analyst,"Business Data Analyst II, Loan Operations, Sou...","South Bend, IN",Full-time,False,"Illinois, United States",2023-04-26 21:03:09,c
773932,99997,1st Source Bank,970738,Data Analyst,"Credit Business Data Analyst I, South Bend, IN","South Bend, IN",Full-time,False,"Illinois, United States",2023-01-14 02:04:10,
773933,99999,Iceland,1033637,Senior Data Analyst,Senior Customer Data Analyst. Job in Deeside M...,"Deeside, UK",Full-time,False,United Kingdom,2023-03-15 10:13:59,sql
773934,99999,Iceland,1035909,Business Analyst,Junior Business Analyst,Austria,Full-time,False,Austria,2023-10-08 21:43:02,
